# Packages

In [1]:
using CSV

In [2]:
using Statistics

In [3]:
using DataFrames

In [4]:
using WebIO

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-5381368097577635263\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-2062256051119373151\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-5381368097577635263\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [5]:
using PlotlyJS

# Results Visualization

In [6]:
include("optimization_main_functions.jl")

create_pfs

## Global Setup for Processing Data

In this section, we define all the global constants that will be used for the optimization.

In [19]:
# Folders Structure
base_folder = pwd()
results_folder = joinpath(base_folder, "algorithms")

# CSV File Configuration
has_header = true
files_sep = ","
file_extension = "csv"

# Optimization Settings
runs = [1]
nruns = length(runs)
max_evals = 600

## Problem Definition (in the files) 
### Variables
Material = :Material
Diameter = :Bar_Radius

vars_cols = [Material, Diameter]

### Objectives  
Max_Displacement = :Max_Displacement
Cost = :Cost

objs_cols = [Max_Displacement, Cost]

relevant_cols = vcat(vars_cols, objs_cols)

names_mapping = (
     4 => Material,
     5 => Diameter,
     6 => Max_Displacement,
     7 => Cost,
        )

## Multi-Objective Optimization Algorithms
### Metaheuristics
pop_size = 30
metaheuristics = ["NSGAII", "MOEAD", "OMOPSO", "PESA2", "SMPSO"]

# all_algorithms = vcat(metaheuristics)
# n_algorithms = length(all_algorithms)

# ### Model-Based (or metamodel)
metamodels_base = ["RFR", "ETR", "GPR"]
metamodels_strategies = ["SPEA2", "SMPSO", "NSGAII"]
metamodels_algorithms = ["$(b)_$(s)" for b in metamodels_base for s in metamodels_strategies]

all_algorithms = vcat(metaheuristics, metamodels_algorithms)
n_algorithms = length(all_algorithms)

# mb_algorithms = vcat(metamodels_algorithms, ["Training"])
# n_mb_algorithms = length(all_algorithms)

### Filenames with the results 
filenames = ["$(a)_results_0$(r).$(file_extension)" for r in runs for a in all_algorithms]
filenames

# mb_filenames = ["$(a)_results_0$(r).$(file_extension)" for r in runs for a in mb_algorithms]
# mb_filenames

14-element Array{String,1}:
 "NSGAII_results_01.csv"
 "MOEAD_results_01.csv"
 "OMOPSO_results_01.csv"
 "PESA2_results_01.csv"
 "SMPSO_results_01.csv"
 "RFR_SPEA2_results_01.csv"
 "RFR_SMPSO_results_01.csv"
 "RFR_NSGAII_results_01.csv"
 "ETR_SPEA2_results_01.csv"
 "ETR_SMPSO_results_01.csv"
 "ETR_NSGAII_results_01.csv"
 "GPR_SPEA2_results_01.csv"
 "GPR_SMPSO_results_01.csv"
 "GPR_NSGAII_results_01.csv"

## Pareto Front

### Read Files

__All Algorithms__

In [20]:
# Read algorithms  
dfs = load_results(filenames);

In [21]:
dfs = [rename!(df, [map(x->x[2], names_mapping) ...]) for df in dfs];

In [22]:
# Sanity check!!
show(first(dfs[1], 6), allcols=true)

6×4 DataFrame
│ Row │ Material │ Bar_Radius │ Max_Displacement │ Cost    │
│     │ Float64  │ Float64    │ Float64          │ Float64 │
├─────┼──────────┼────────────┼──────────────────┼─────────┤
│ 1   │ 2.0      │ 7.0        │ -0.0187472       │ 3854.12 │
│ 2   │ 6.0      │ 8.0        │ -0.0190831       │ 5894.65 │
│ 3   │ 3.0      │ 15.0       │ -0.0197493       │ 7403.01 │
│ 4   │ 6.0      │ 14.0       │ -0.0197013       │ 12591.5 │
│ 5   │ 4.0      │ 20.0       │ -0.0198616       │ 13346.8 │
│ 6   │ 2.0      │ 4.0        │ -0.0167522       │ 2167.94 │

### Layout

In [36]:
layout_truss = Layout(
    template="plotly_white",
    autosize=false,
    # Define plot size
    width=800, 
    height=700,
    # Legend Position
    # showlegend = False,
    legend=Dict(
        :orientation=>'h',
        :x=>-0.01,
        :y=>-0.2,
        :show=>false
    ),

    # Define axis
    xaxis=Dict(
#         :title=>"Maximum Displacement [m]",
        :title=>"Cost [€]",
        :autorange=>true,
#         :range=>[3000, 12000],
        :type=>"log",
#         :showgrid=>true,
#         :zeroline=>false,
#         :showline=>true,
#         :ticks=>"",
#         :showticklabels=>true,
#         :tickformat=>"."
    ),
  
    yaxis=Dict(
#         :title=>"Cost [€]",
        :title=>"Maximum Displacement [m]",
        :autorange=>true, 
#         :range=>[0, 0.015],
#         :type=>"log",
#         :showgrid=>true,
#         :zeroline=>false,
#         :showline=>true,
#         :ticks=>"",
#         :showticklabels=>true,
#         :tickformat=>"."
    ),
    anotations=Dict(
        :x=>-5,
        :y=>-5
    )
)

layout with fields anotations, autosize, height, legend, margin, template, width, xaxis, and yaxis


### Compute Non-Dominated and Dominated Solutions

__All Algorithms__

In [24]:
# Compute non_dominated_solutions (per run)
pfs = [add_isdominated_cols(df) for df in dfs];

2×2 DataFrame
│ Row │ isDominated │ x1    │
│     │ Float64     │ Int64 │
├─────┼─────────────┼───────┤
│ 1   │ 1.0         │ 290   │
│ 2   │ 0.0         │ 310   │
2×2 DataFrame
│ Row │ isDominated │ x1    │
│     │ Float64     │ Int64 │
├─────┼─────────────┼───────┤
│ 1   │ 1.0         │ 300   │
│ 2   │ 0.0         │ 300   │
2×2 DataFrame
│ Row │ isDominated │ x1    │
│     │ Float64     │ Int64 │
├─────┼─────────────┼───────┤
│ 1   │ 1.0         │ 433   │
│ 2   │ 0.0         │ 167   │
2×2 DataFrame
│ Row │ isDominated │ x1    │
│     │ Float64     │ Int64 │
├─────┼─────────────┼───────┤
│ 1   │ 1.0         │ 375   │
│ 2   │ 0.0         │ 225   │
2×2 DataFrame
│ Row │ isDominated │ x1    │
│     │ Float64     │ Int64 │
├─────┼─────────────┼───────┤
│ 1   │ 1.0         │ 437   │
│ 2   │ 0.0         │ 163   │
2×2 DataFrame
│ Row │ isDominated │ x1    │
│     │ Float64     │ Int64 │
├─────┼─────────────┼───────┤
│ 1   │ 1.0         │ 376   │
│ 2   │ 0.0         │ 224   │
2×2 DataFrame
│ 

In [25]:
# Sanity check!!
first(pfs[1], 6)

,Material,Bar_Radius,Max_Displacement,Cost,isDominated,dominatedBy
,Float64,Float64,Float64,Float64,Float64,Float64
1,2.0,7.0,-0.0187472,3854.12,1.0,23.0
2,6.0,8.0,-0.0190831,5894.65,1.0,10.0
3,3.0,15.0,-0.0197493,7403.01,1.0,19.0
4,6.0,14.0,-0.0197013,12591.5,1.0,3.0
5,4.0,20.0,-0.0198616,13346.8,0.0,0.0
6,2.0,4.0,-0.0167522,2167.94,1.0,24.0


In [26]:
# Computes combined Pareto Front (optimal solutions found from all the algorithms, all the runs)
combined_pf = get_combined_PF(dfs, drop_cols=relevant_cols);

2×2 DataFrame
│ Row │ isDominated │ x1    │
│     │ Float64     │ Int64 │
├─────┼─────────────┼───────┤
│ 1   │ 1.0         │ 101   │
│ 2   │ 0.0         │ 25    │


In [27]:
# Sanity check!!
first(combined_pf, 6)

,Material,Bar_Radius,Max_Displacement,Cost,isDominated,dominatedBy
,Float64,Float64,Float64,Float64,Float64,Float64
1,2.0,7.0,-0.0187472,3854.12,1.0,21.0
2,6.0,8.0,-0.0190831,5894.65,1.0,10.0
3,3.0,15.0,-0.0197493,7403.01,1.0,18.0
4,6.0,14.0,-0.0197013,12591.5,1.0,3.0
5,4.0,20.0,-0.0198616,13346.8,0.0,0.0
6,2.0,4.0,-0.0167522,2167.94,1.0,22.0


In [28]:
# Sort Cost values in ascending order

## This fixes the error we were getting in the create_pfs_interactive function,
## which would return a different point than the one we selected

pfs = [sort(pf, [Cost], rev=false) for pf in pfs]
combined_pf = sort(combined_pf, [Cost], rev=false)
nothing

In [29]:
# Sanity check!!
first(pfs[1], 6)

,Material,Bar_Radius,Max_Displacement,Cost,isDominated,dominatedBy
,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,0.0,0.0936975,327.441,0.0,0.0
2,1.0,0.0,0.0936975,327.441,0.0,0.0
3,1.0,0.0,0.0936975,327.441,0.0,0.0
4,1.0,0.0,0.0936975,327.441,0.0,0.0
5,1.0,0.0,0.0936975,327.441,0.0,0.0
6,1.0,0.0,0.0936975,327.441,0.0,0.0


## Pareto Front Plot

### All Algorithms x 1 Front per Run

In [37]:
plot = create_pfs(pfs, y=Max_Displacement, x=Cost, tpf=nothing, names=filenames, 
           colorscale=nothing,
    colors=["#43a0b5", "#B5557A", "#eb911c", 
            "#fc0384", "#3150d6", "#32a852", 
            "#9b60d1", "#fc0384", "#3150d6",
            "#8b32a8", "#5996cf", "#702743",
            "#43a0b5", "#B5557A", "#eb911c", 
            "#fc0384", "#3150d6", "#32a852", 
            "#9b60d1", "#fc0384", "#3150d6",
            "#8b32a8", "#5996cf", "#702743",
            "#43a0b5", "#B5557A", "#eb911c", 
            "#fc0384", "#3150d6", "#32a852", 
            "#9b60d1", "#fc0384", "#3150d6",
            "#8b32a8", "#5996cf", "#702743"],
           draw_dominated=true, layout=layout_truss)

data: [
  "scatter with fields line, marker, mode, name, opacity, type, x, and y",
  "scatter with fields marker, mode, name, opacity, type, x, and y",
  "scatter with fields line, marker, mode, name, opacity, type, x, and y",
  "scatter with fields marker, mode, name, opacity, type, x, and y",
  "scatter with fields line, marker, mode, name, opacity, type, x, and y",
  "scatter with fields marker, mode, name, opacity, type, x, and y",
  "scatter with fields line, marker, mode, name, opacity, type, x, and y",
  "scatter with fields marker, mode, name, opacity, type, x, and y",
  "scatter with fields line, marker, mode, name, opacity, type, x, and y",
  "scatter with fields marker, mode, name, opacity, type, x, and y",
  "scatter with fields line, marker, mode, name, opacity, type, x, and y",
  "scatter with fields marker, mode, name, opacity, type, x, and y",
  "scatter with fields line, marker, mode, name, opacity, type, x, and y",
  "scatter with fields marker, mode, name, opacity, type, x, and y",
  "scatter with fields line, marker, mode, name, opacity, type, x, and y",
  "scatter with fields marker, mode, name, opacity, type, x, and y",
  "scatter with fields line, marker, mode, name, opacity, type, x, and y",
  "scatter with fields marker, mode, name, opacity, type, x, and y",
  "scatter with fields line, marker, mode, name, opacity, type, x, and y",
  "scatter with fields marker, mode, name, opacity, type, x, and y",
  "scatter with fields line, marker, mode, name, opacity, type, x, and y",
  "scatter with fields marker, mode, name, opacity, type, x, and y",
  "scatter with fields line, marker, mode, name, opacity, type, x, and y",
  "scatter with fields marker, mode, name, opacity, type, x, and y",
  "scatter with fields line, marker, mode, name, opacity, type, x, and y",
  "scatter with fields marker, mode, name, opacity, type, x, and y",
  "scatter with fields line, marker, mode, name, opacity, type, x, and y",
  "scatter with fields marker, mode, name, opacity, type, x, and y"
]

layout: "layout with fields anotations, autosize, height, legend, margin, template, width, xaxis, and yaxis"

### All Algorithms x 1 Front per Algorithm

In [ ]:
# Get Run 1 
dfs_run_1 = get_run_indices(dfs, 1)
pfs_run_1 = [add_isdominated_cols(df) for df in dfs_run_1]
combined_PF_run_1 = get_combined_PF(pfs_run_1)

In [ ]:
# Get Run 2 
dfs_run_2 = get_run_indices(dfs, 2)
pfs_run_2 = [add_isdominated_cols(df) for df in dfs_run_2]
combined_PF_run_2 = get_combined_PF(pfs_run_2)

In [ ]:
# Get Run 3 
dfs_run_3 = get_run_indices(dfs, 3)
pfs_run_3 = [add_isdominated_cols(df) for df in dfs_run_3]
combined_PF_run_3 = get_combined_PF(pfs_run_3)

In [ ]:
# Get All Runs
# Before running this run the previous cells (the ones that define dfs_run_{i})
pfs_all_runs = []
for i in 1:n_algorithms
    pf=get_combined_PF([dfs_run_1[i], dfs_run_2[i], dfs_run_3[i]])
    pfs_all_runs = vcat(pfs_all_runs, pf)
end

In [ ]:
plot=create_pfs(pfs_all_runs, y=Max_Displacement, x=Cost, tpf=nothing, names=all_algorithms, 
           colorscale=nothing,
#     colors=["#add4f1", "#2d85c6", # PSOs colors
#             "#581822", "#b03144", "#b74556", "#dfacb4", # EAs colors 
#             "#7e497b", "#d687d2", "#f1d7f0", # MBs colors
#            ],
        colors=["#ffb300", "#FF8C61", #PSOs colors
            "#994b45", "#d96483", "#c78ba1", "#5c374c", # EAs colors
            "#11706d", "#42c7c3", "#a0e0e8"], # MBs colors

    draw_dominated=true, layout=layout_truss)

In [ ]:
# Saving plots -> does not work

using Plots; plotlyjs()

PlotlyJS.savefig(plot, "pareto_asa.pdf")

In [ ]:
using ORCA

In [ ]:
PlotlyJS.savefig(plot, "pareto_asa.pdf", format="pdf")

In [ ]:
pwd()

In [ ]:
import Pkg
Pkg.add("Plots")